In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from glob import glob

In [7]:
def is_consecutive(days):
    if days>=1 and days<=3:
        return True
    else:
        return None

In [8]:
def EHF(df):
    p95 = df['temperature_mean'].quantile(0.9)
    data = list()
    for i in range(29, len(df['temperature_mean'])):
        try:
            time = df.index[i+3]
            temperature_mean = df['temperature_mean'].iloc[i+3]
            temperature_min = df['temperature_min'].iloc[i+3]
            temperature_max = df['temperature_max'].iloc[i+3]
            last3days = df['temperature_mean'].iloc[i:i+3]
            last30days = df['temperature_mean'].iloc[i-29:i]
            
            ehfsig = last3days.mean() - p95
            ehfacl = last3days.mean() - last30days.mean()
        
            #print(i)
            #print((i,i+3))
            #print(last3days)
            #print((i-29,i))
            #print(last30days)
            #print(time)
            
            EHF = max(0, ehfsig) * max(1, ehfacl)
            data.append({
                'time':time, 'temperature_mean':temperature_mean, 'temperature_min':temperature_min, 'temperature_max':temperature_max, 'ehfsig':ehfsig, 'ehfacl':ehfacl, 'EHF':EHF
            })

        except:
            pass
    df_ehf = pd.DataFrame(data)
    df_ehf.index = df_ehf.time
    df_ehf['severity'] = df_ehf[df_ehf.EHF>0]['EHF'] / df_ehf[df_ehf.EHF>0]['EHF'].quantile(0.85)
    return df_ehf

In [20]:
df_anom = pd.read_csv("data/pos/tmean_anomaly_groups_kmeans.csv", index_col=0)
df_clim = pd.read_csv("data/pos/tmean_clim_groups_kmeans.csv", index_col=0)
df_tmean = pd.read_csv("data/pos/tmean_groups_kmeans.csv", index_col=0)
citys = pd.read_csv("data/statics/cidades.csv", index_col=2).drop(columns=['Unnamed: 0'])

In [21]:
citys

,names,nans,lat,lon
codes,,,,
83836,IRATI,4.0,-25.502778,-50.637500
83912,CRUZALTA,5.6,-28.626111,-53.613889
83964,ENCRUZILHADADOSUL,4.5,-30.543056,-52.524722
83948,TORRES,7.6,-29.350278,-49.716667
83919,BOMJESUS,2.5,-28.669444,-50.441944
83980,BAGE,1.5,-31.305556,-54.119444
83967,PORTOALEGRE,2.8,-30.053611,-51.174767
83766,LONDRINA,9.2,-23.325556,-51.141667
83914,PASSOFUNDO,1.7,-28.228611,-52.411944


In [22]:
df_tmean

,G1,G2,G3,G4
1991-01-01,22.30,21.250000,24.350000,23.620000
1991-01-02,23.80,22.100000,24.583333,26.200000
1991-01-03,23.95,20.766667,22.083333,25.500000
1991-01-04,22.95,21.566667,20.983333,22.933333
1991-01-05,22.85,20.366667,21.516667,24.600000
...,...,...,...,...
2020-12-27,24.00,23.333333,24.500000,21.875000
2020-12-28,24.30,23.525000,24.620000,21.820000
2020-12-29,24.60,22.000000,25.280000,23.620000
2020-12-30,25.30,24.325000,26.080000,24.560000


In [19]:
for code in citys.index:
    print

83836


In [9]:
df = pd.read_csv('./1991_2020.csv',index_col=0,usecols=[0, 4, 5, 6, 10, 11, 12, 13])
#df = df[df['latitude'] <= -23.325556]
df.index = pd.to_datetime(df.index)
df = df.sort_index().dropna()
df['temperature_mean'] = df[['temperature_max', 'temperature_min']].mean(axis=1)
for station_name in df.station_name.unique():
    df_poa = df[df.station_name == station_name]
    df_EHF = EHF(df_poa)
    df_EHF = df_EHF[df_EHF['EHF']>0]
    name = '_'.join(station_name.split(' '))
    path_out = f'./EHF_{name}.csv'
    print(path_out)
    df_EHF.to_csv(path_out)
# times = df_EHF.index
# heatwaves = list()
# periods = list()

# c = 0
# for i, timenow in enumerate(times):
#     if i == 0:
#         pass
#     else:
#         days = (timenow - times[i-1]).days
#         if is_consecutive(days):
#             heatwaves.append(timenow)
#             c += 1
#         else:
#             if c>=1:
#                 periods.append(heatwaves)
#             c = 0
#             heatwaves = list()

./EHF_PORTO_ALEGRE.csv
./EHF_ENCRUZILHADA_DO_SUL.csv
./EHF_SANTA_MARIA.csv
./EHF_MARINGA.csv
./EHF_FLORIANOPOLIS.csv
./EHF_SAO_LUIZ_GONZAGA.csv
./EHF_TORRES.csv
./EHF_CRUZ_ALTA.csv
./EHF_PELOTAS.csv
./EHF_BOM_JESUS.csv
./EHF_URUGUAIANA.csv
./EHF_BAGE.csv
./EHF_CURITIBA.csv
./EHF_PASSO_FUNDO.csv
./EHF_SANTA_VITORIA_DO_PALMAR.csv
./EHF_IRATI.csv
./EHF_LONDRINA.csv
./EHF_CAXIAS_DO_SUL.csv


,time,temperature_mean,temperature_min,temperature_max,ehfsig,ehfacl,EHF,severity
time,,,,,,,,
1991-02-11,1991-02-11,29.45,22.4,36.5,0.066667,2.202874,0.146858,0.018655
1991-02-12,1991-02-12,24.10,21.0,27.2,1.116667,3.239080,3.616973,0.459466
1991-02-13,1991-02-13,22.45,17.4,27.5,0.333333,2.354023,0.784674,0.099678
1991-03-01,1991-03-01,27.30,21.4,33.2,1.250000,3.537931,4.422414,0.561782
1991-03-02,1991-03-02,27.10,22.7,31.5,1.383333,3.531609,4.885393,0.620594
...,...,...,...,...,...,...,...,...
2020-04-01,2020-04-01,25.60,21.8,29.4,0.233333,1.609195,0.375479,0.047697
2020-11-26,2020-11-26,24.20,20.5,27.9,0.016667,5.495977,0.091600,0.011636
2020-11-27,2020-11-27,23.40,21.1,25.7,0.133333,5.681609,0.757548,0.096232


In [141]:
periods

[[Timestamp('1991-02-12 00:00:00'), Timestamp('1991-02-13 00:00:00')],
 [Timestamp('1991-03-02 00:00:00'),
  Timestamp('1991-03-03 00:00:00'),
  Timestamp('1991-03-04 00:00:00')],
 [Timestamp('1991-03-10 00:00:00'), Timestamp('1991-03-11 00:00:00')],
 [Timestamp('1991-10-17 00:00:00')],
 [Timestamp('1991-11-26 00:00:00')],
 [Timestamp('1991-12-12 00:00:00')],
 [Timestamp('1991-12-20 00:00:00'),
  Timestamp('1991-12-21 00:00:00'),
  Timestamp('1991-12-22 00:00:00'),
  Timestamp('1991-12-23 00:00:00'),
  Timestamp('1991-12-24 00:00:00'),
  Timestamp('1991-12-25 00:00:00')],
 [Timestamp('1992-01-12 00:00:00')],
 [Timestamp('1992-01-30 00:00:00'),
  Timestamp('1992-01-31 00:00:00'),
  Timestamp('1992-02-01 00:00:00'),
  Timestamp('1992-02-02 00:00:00'),
  Timestamp('1992-02-03 00:00:00'),
  Timestamp('1992-02-04 00:00:00'),
  Timestamp('1992-02-06 00:00:00'),
  Timestamp('1992-02-07 00:00:00'),
  Timestamp('1992-02-08 00:00:00'),
  Timestamp('1992-02-09 00:00:00'),
  Timestamp('1992-02-10 